In [55]:
import sys
sys.path.append('../')
import numpy as np
import xpress as xp
import pandas as pd
from itertools import chain
from config.config import (COMPETITORS,
                           FIRST_WEEK,
                           MAX_CONVERSION_RATE,
                           SLOT_DURATION,
                           TOTAL_SLOTS,
                           WEEK_CONSIDERED,
                           YEAR,
                            DEMOGRAPHIC_LIST,
                            TOTAL_VIEW_COUNT
                            )
from utils.data_processing import (process_table,
                                   DEMOGRAPHIC_LIST,
                                   top_n_viable_film)
from utils.schedule_processing import (combine_schedule,
                                       consolidate_time_to_30_mins_slot,
                                       dynamic_pricing,
                                       return_selected_week,
                                       get_date_from_week,
                                       create_competitor_schedule,
                                       decay_view_penelty,
                                       process_current_week,
                                       process_competitor_current_week,
                                       update_schedule,
                                       return_ads_30_mins,
                                       sort_df_by_slot_day,
                                       de_one_hot_columns_include_empty_slots)
from datetime import datetime as dt
from utils.advert_conversion_rates import generate_conversion_rates
def import_data():
    base_path = "../data/"

    mov_df = pd.read_csv(base_path + 'movie_database.csv')

    ch_0_conversion_rates_df = pd.read_csv(base_path + 'channel_0_conversion_rates.csv', index_col=[0])
    ch_0_conversion_rates_df.index = pd.to_datetime(ch_0_conversion_rates_df.index)

    ch_1_conversion_rates_df = pd.read_csv(base_path + 'channel_1_conversion_rates.csv', index_col=[0])
    ch_1_conversion_rates_df.index = pd.to_datetime(ch_1_conversion_rates_df.index)

    ch_2_conversion_rates_df = pd.read_csv(base_path + 'channel_2_conversion_rates.csv', index_col=[0])
    ch_2_conversion_rates_df.index = pd.to_datetime(ch_2_conversion_rates_df.index)

    ch_a_schedule_df = pd.read_csv(base_path + 'channel_A_schedule.csv', index_col=[0])
    ch_a_schedule_df.index = pd.to_datetime(ch_a_schedule_df.index)

    ch_0_schedule_df = pd.read_csv(base_path + 'channel_0_schedule.csv', index_col=[0])
    ch_0_schedule_df.index = pd.to_datetime(ch_0_schedule_df.index)

    ch_1_schedule_df = pd.read_csv(base_path + 'channel_1_schedule.csv', index_col=[0])
    ch_1_schedule_df.index = pd.to_datetime(ch_1_schedule_df.index)

    ch_2_schedule_df = pd.read_csv(base_path + 'channel_2_schedule.csv', index_col=[0])
    ch_2_schedule_df.index = pd.to_datetime(ch_2_schedule_df.index)

    return (mov_df, ch_0_conversion_rates_df, ch_1_conversion_rates_df, ch_2_conversion_rates_df, ch_a_schedule_df,
            ch_0_schedule_df, ch_1_schedule_df, ch_2_schedule_df)


print("==== Starting Importing Data ====")
# Import Data
st = dt.now()
(movie_df, channel_0_conversion_rates_df, channel_1_conversion_rates_df, channel_2_conversion_rates_df,
 channel_a_schedule_df, channel_0_schedule_df, channel_1_schedule_df, channel_2_schedule_df) = import_data()
print("===== Total time used to import data: {0} seconds".format((dt.now() - st).total_seconds()))



==== Starting Importing Data ====
===== Total time used to import data: 0.419759 seconds


In [2]:
mov_df, ch_0_conversion_rates_df, ch_1_conversion_rates_df, ch_2_conversion_rates_df, ch_a_schedule_df, ch_0_schedule_df, ch_1_schedule_df, ch_2_schedule_df = import_data()

In [3]:
movie_df = process_table(mov_df)
original_movie_df = movie_df.copy()

# Create DF needed in the models
competitor_schedules = [channel_0_schedule_df, channel_1_schedule_df, channel_2_schedule_df]
channel_a_30_schedule_df = consolidate_time_to_30_mins_slot(channel_a_schedule_df)

/home/punny/studying/mmcs/mmcs-project-uoe/data_exploration/../utils/data_processing.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_df[f'total_expected_view_count'] = total_view_count
/home/punny/studying/mmcs/mmcs-project-uoe/data_exploration/../utils/data_processing.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_df['ad_slot_price'] = ad_slot_price(movie_df['license_fee'],
/home/punny/studying/mmcs/mmcs-project-uoe/data_exploration/../utils/data_processing.py:74: SettingWithCopyWarn

In [75]:
week = 41

In [76]:
week_40_mv = pd.read_csv("../out/week_41/movie_time.csv")
week_40_iv = pd.read_csv("../out/week_41/bought_ad_slots.csv")

In [77]:
schedule_df = sort_df_by_slot_day(week_40_mv)
schedule_df = de_one_hot_columns_include_empty_slots(schedule_df, channel_a_30_schedule_df.index)
schedule_df = schedule_df.set_index('time')

In [78]:
ch_a_schedule_df = consolidate_time_to_30_mins_slot(channel_a_schedule_df)
ch_a_schedule_df = return_selected_week(ch_a_schedule_df, week)

In [80]:
ads_price_per_view = dynamic_pricing(
            week=week, competitor_schedule_list=competitor_schedules)

In [82]:
sum_df = pd.DataFrame()
for demo in DEMOGRAPHIC_LIST:
    sum_df[f'{demo}'] = all_df[f'{demo}_scaled_popularity'] * ch_a_schedule_df[
    'prime_time_factor'].values * ch_a_schedule_df[
    f'{demo}_baseline_view_count'].values * ads_price_per_view * TOTAL_VIEW_COUNT

In [86]:
base_view = sum_df.sum(axis=1).sum()

In [87]:
licensing_fee = all_df.drop_duplicates(subset='content')[['license_fee']].sum()

In [85]:
base_view - licensing_fee

license_fee    1.518835e+07
dtype: float64

['children_scaled_popularity', 'adults_scaled_popularity', 'retirees_scaled_popularity']


ValueError: Columns must be same length as key

In [19]:
temp_sum

,adults_scaled_popularity,children_scaled_popularity,retirees_scaled_popularity
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
5915,NaN,NaN,NaN
5916,NaN,NaN,NaN
5917,NaN,NaN,NaN
5918,NaN,NaN,NaN


In [ ]:
all_df[['total_scaled_popularity', 'license_fee']]

In [ ]:
movie_df

In [ ]:
schedule_df

In [ ]:
schedule_df['license_fee']

In [ ]:
schedule_df

In [ ]:
week_40_iv

In [99]:
movie_df[demo_columns].sum(axis=1)

0       2.138460
1       2.200000
2       2.400000
3       2.108039
4       2.200000
          ...   
5915    0.160167
5916    0.000003
5917    0.000003
5918    0.000002
5919    0.202456
Length: 5920, dtype: float64

In [102]:
demo_columns = [f'{demo}_scaled_popularity' for demo in DEMOGRAPHIC_LIST]
demo_df = movie_df.loc[:,  demo_columns]
max_view = demo_df.max(axis=1).to_numpy()
movie_df['total_scaled_popularity'] = movie_df[demo_columns].sum(axis=1)
mean_view = movie_df.loc[:, ['total_scaled_popularity']].to_numpy().flatten()
# licensing_fee = movie_df['license_fee'].to_numpy().flatten()

In [104]:
licensing_fee = movie_df['license_fee'].to_numpy().flatten()

KeyError: 'license_fee'

In [103]:
mean_view

array([2.1384604e+00, 2.2000000e+00, 2.4000000e+00, ..., 3.0000000e-06,
       2.4000000e-06, 2.0245560e-01])